In [ ]:
import cv2 as cv
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dense, MaxPooling2D, Dropout, Flatten, GlobalAveragePooling2D
from matplotlib.image import imread
import matplotlib.pyplot as plt
import random
from resnet50 import ResNet50

In [ ]:
training_dir = "./data/chest-ctscan-images/train"
validation_dir = "./data/chest-ctscan-images/valid"
testing_dir = "./data/chest-ctscan-images/test"

In [ ]:
def display_random_images(directory_path):
    image_files = os.listdir(directory_path)
    fig, ax = plt.subplots(2, 5, figsize=(12, 4))

    for i in range(2):
        for j in range(5):
            img_path = os.path.join(directory_path, random.choice(image_files))
            image = imread(img_path)
            ax[i, j].imshow(image, cmap="gray")
            ax[i, j].axis('off')

    plt.show()

train_images_path = training_dir + '/normal'
display_random_images(train_images_path)

# Image augmentation

In [ ]:
INPUT_SHAPE = (224, 224, 3)
NUM_CLASSES = 4

train_data_augmentor = ImageDataGenerator(rotation_range=10,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          vertical_flip=False,
                                          preprocessing_function=preprocess_input,
                                          dtype="float32")

test_data_augmentor = ImageDataGenerator(preprocessing_function=preprocess_input,
                                        dtype="float32")

val_data_augmentor = ImageDataGenerator(preprocessing_function=preprocess_input,
                                        dtype="float32")

In [ ]:
train_data = train_data_augmentor.flow_from_directory(training_dir,
                                                      target_size=(224, 224),
                                                      batch_size=32,
                                                      class_mode="categorical")

test_data = test_data_augmentor.flow_from_directory(testing_dir,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode="categorical")

val_data = val_data_augmentor.flow_from_directory(validation_dir,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode="categorical")

In [ ]:
train_data.class_indices

#  Resnet model

In [ ]:
resnet_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
resnet_model.summary()

# Transfer learning

In [ ]:
for layer in resnet_model.layers:
    layer.trainable = False

custom_model = Sequential()
custom_model.add(resnet_model)
custom_model.add(BatchNormalization())
custom_model.add(MaxPooling2D(pool_size=(2, 2)))
custom_model.add(Dropout(0.3))
custom_model.add(Flatten())
custom_model.add(Dense(1024, activation="relu"))
custom_model.add(Dropout(0.3))
custom_model.add(Dense(512, activation="relu"))
custom_model.add(Dropout(0.3))
custom_model.add(Dense(256, activation="relu"))
custom_model.add(Dropout(0.3))
custom_model.add(Dense(NUM_CLASSES, activation='softmax'))
custom_model.summary()

# Compiling the model

In [ ]:
optimizer = Adam(learning_rate=0.001)
custom_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Training the model

In [ ]:
model_checkpoint = ModelCheckpoint(
    filepath="ct_resnet_best_model.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,
    patience=5,
    verbose=1,
    min_delta=0.8
)

training_history = custom_model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    batch_size=32,
    callbacks=[model_checkpoint, early_stopping, reduce_lr]
)

In [ ]:
plt.plot(training_history.history['accuracy'])
plt.plot(training_history.history['val_accuracy'])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

# Test 

In [ ]:
saved_model = load_model("ct_resnet_best_model.hdf5")
test_scores = saved_model.evaluate(test_data, verbose=1)

print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])